In [ ]:
"""
Task 3. Dataset 2 
    Input -
        Datasets_for_A1 folder needs to be in the current working 
        directory with the folder structure as 
        Datasets_for_A1/Regression/Dataset 2/<team number>/
        Replace <team number> with the actual number.
        e.g. Datasets_for_A1/Regression/Dataset 2/9/
    Output -
        Linear model for regression using Gaussian basis functions for Datasets 2 and 3
        Regularization methods:
            1. No regularization
            2. Quadratic regularization
        Presentation of Results:
            • Scatter plots with target output on x-axis and model output on y-axis 
            for the best performing model, for training data and test data.
            • Tables showing the 𝐸𝑅𝑀𝑆 on the training data, the validation
            data and the test data, for different models.
"""
import os
import warnings
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

lambda_string = "λ"
column_names = [
    "Index",
    "Sample Size",
    "λ",
    "Train Erms",
    "Validate Erms",
    "Test Erms",
]
table_index = 0
table_df = pd.DataFrame(columns=column_names)


def add_data_to_table(sample_size, lamda, train_erms, validate_erms, test_erms):
    row_data = {}
    global table_df
    global table_index
    table_index = table_index + 1
    row_data["Index"] = table_index
    row_data["Sample Size"] = sample_size
    row_data["λ"] = lamda
    row_data["Train Erms"] = train_erms
    row_data["Validate Erms"] = validate_erms
    row_data["Test Erms"] = test_erms
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        table_df = pd.concat(
            [table_df, pd.DataFrame(row_data, index=[0])], ignore_index=True
        )


def print_table():
    print(table_df.to_string(index=False))


def get_N(x):
    N = len(x)
    return N


def get_identity_matrix(n):
    identity_matrix = np.eye(n)

    return identity_matrix

def estimate_sigma(x, centroids):
    sigma_list = []
    # x1=x[0]
    # x2=x[1]
    # data = np.column_stack((x1, x2))
    data = x
    N=data.shape[0]
    for loop in range(N):
        small_phi = []
        data_x = data[loop]
        # print("data_x",data_x)
        for centroid in centroids:
            # print("centroid",type(centroid))
            # centroidn = np.array(centroid)
            # print("centroidn",type(centroidn))
            sum = np.square(np.linalg.norm(data_x-centroid,2))
            sigma_list.append(sum)
    sigma_avg = np.average(sigma_list)

    return sigma_avg


def get_phi_by_gaussian(x, D):
    # D = len(train_x_11) * 0.2
    # D = int(D)
    # print(len(x))
    # print("x",x)
    # x1=x[0]
    # x2=x[1]
    # data = np.column_stack((x1, x2))
    data = x
    # print(data)
    # data = x
    cluster_count = D - 1

    indices = list(range(data.shape[0]))
    # print("indices",indices)
    random.shuffle(indices)
    centroid_indices = indices[:cluster_count]
    # print("centroid_indices",centroid_indices)
    centroid_indices = np.array(centroid_indices)
    # print(centroid_indices)

    centroids = data[centroid_indices]
    # print("centroids",centroids)

    cluster_members = [[] for _ in range(cluster_count)]

    while True:
        distances = np.empty((data.shape[0], centroids.shape[0]))
        # print("distances",distances)
        for i, point in enumerate(data):
            for j, centroid in enumerate(centroids):
                distance = np.linalg.norm(point-centroid,2)
                distances[i, j] = distance

        # print("distances",distances)

        labels = []

        for row in distances:
            min_distance = float("inf")
            min_index = -1
            # print("min_distance ",min_distance," min_index ",min_index)
            for i, distance in enumerate(row):
                if distance < min_distance:
                    min_distance = distance
                    min_index = i
                    # print("min_distance ",min_distance," min_index ",min_index)
            labels.append(min_index)

        labels = np.array(labels)

        # print("labels",labels)

        new_centroids = []

        for k in range(cluster_count):
            cluster_points = data[labels == k]
            # print("cluster_points",cluster_points)
            centroid = cluster_points.mean(axis=0)
            # print("centroid",centroid)
            new_centroids.append(centroid)

        new_centroids = np.array(new_centroids)

        # print("centroids",centroids)
        # print("new_centroids",new_centroids)

        cluster_members = [[] for _ in range(cluster_count)]
        centroid_mapping = {tuple(centroid): [] for centroid in centroids}
        for i, label in enumerate(labels):
            cluster_members[label].append(data[i])
            centroid_mapping[tuple(centroids[label])].append(data[i])
        
        # print("centroid_mapping",centroid_mapping)
        # print("===================================================")
        # for centroid, members in centroid_mapping.items():
        #     print(f"Centroid: {centroid}, Cluster Members: {members}")
        # print("===================================================")
        


        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids
        

    # print("centroid_mapping",centroid_mapping)
    # print("centroids",centroids)
    # print("centroids",centroids[0])

    # plt.figure(figsize=(8, 6))

    # for centroid, members in centroid_mapping.items():
    #     print(f"Centroid: {centroid}, Cluster Members: {members}")

    # print("cluster_members",cluster_members)
    # centroid_colors = plt.cm.Dark2(np.linspace(0, 1, cluster_count+1))
    # k=0
    # for i, (centroid, members) in enumerate(centroid_mapping.items()):
    #     cluster_points = np.array(members)
    #     # print("cluster_points",cluster_points)
    #     data_x1, data_x2 = cluster_points[:, 0], cluster_points[:, 1]
    #     with warnings.catch_warnings():
    #         warnings.simplefilter("ignore")
    #         plt.scatter(data_x1, data_x2, label=f'Cluster {k+1}',c=centroid_colors[k])
    #         # plt.scatter(data_x1, data_x2, label=f'Cluster {k+1}',cmap='Dark2')
    #     centroidn = np.array(centroid)
    #     centroids_x1, centroids_x2 = centroidn[0], centroidn[1]
    #     with warnings.catch_warnings():
    #         warnings.simplefilter("ignore")
    #         plt.scatter(centroids_x1, centroids_x2, marker='x', s=100, c=centroid_colors[k])
    #         # plt.scatter(centroids_x1, centroids_x2, marker='x', s=100, c=centroid_colors[k])
    #     k=k+1

    # plt.xlabel('x1')
    # plt.ylabel('x2')
    # plt.title('K-means Clustering')
    # plt.legend()
    # plt.show()

    ''''
    kmean clustering end
    '''
    return centroids


def get_big_phi(x, centroids, sigma):
    # big_phi = []
    # x1 = x[0]
    # x2 = x[1]
    # big_phi = get_phi_by_gaussian(x1, x2, D)
    # # print("design matrix ", big_phi)
    # return big_phi

    big_phi = []
    # for members in cluster_members:
    # cluster_points = np.array(members)
    # data_x1, data_x2 = cluster_points[:, 0], cluster_points[:, 1]
    # print("data.shape=",data.shape)
    # x1=x[0]
    # x2=x[1]
    # data = np.column_stack((x1, x2))
    data = x
    # print("data",data)
    N=data.shape[0]
    # print("N",N)
    for loop in range(N):
        small_phi = []
        data_x = data[loop]
        # print("data_x",data_x)
        for centroid in centroids:
            # print("centroid",type(centroid))
            # centroidn = np.array(centroid)
            # print("centroidn",type(centroidn))
            sum = np.square(np.linalg.norm(data_x-centroid,2))
            sum = sum / sigma
            sum = np.exp(-sum)
            small_phi.append(sum)
        small_phi = np.insert(small_phi, 0, 1)
        big_phi.append(small_phi)

    np_big_phi = np.array(big_phi)
    # print("big_phi",np_big_phi)

    return np_big_phi


def get_new_y(w, big_phi, N, D):
    new_y = []
    loop1 = 0

    for loop1 in range(N):
        small_phi = big_phi[loop1]
        # print("small phi ",small_phi)
        y = 0
        loop2 = 0
        for loop2 in range(D):
            wi = w[loop2]
            small_phii = small_phi[loop2]
            y = y + (wi * small_phii)
        new_y.append(y)
    # print("new_y", new_y)
    np_new_y = np.array(new_y)
    # print("new_y", np_new_y)
    return np_new_y


def get_erms(y, t):
    loop = 0
    total_data = len(y)
    # print("len y ", len(y), " len t ", len(t))
    sum = 0

    for loop in range(total_data):
        sum = sum + np.square(y[loop] - t[loop])
        # print("loop ", loop, " sum ", sum)

    error = sum / total_data

    erms = np.sqrt(error)

    return erms


def scatter_plot_2d(x, y, x_label, y_label, plot_title, plot_color):
    plt.scatter(x, y, color=plot_color, label=plot_title)
    plt.title(plot_title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    # plt.show()


def plot_scatter(
    train_y, new_train_y, val_y, new_val_y, test_y, new_test_y, title, sample_size, lamda
):
    plot_title = "Train Data Scatter Plot"
    scatter_plot_2d(train_y, new_train_y, "Actual", "Predicted", plot_title, "blue")

    plot_title = "Validation Data Scatter Plot"
    scatter_plot_2d(val_y, new_val_y, "Actual", "Predicted", plot_title, "green")

    plot_title = "Test Data Scatter Plot"
    scatter_plot_2d(test_y, new_test_y, "Actual", "Predicted", plot_title, "red")

    plot_title = title + "ScatterPlot - SampleSize=" + str(sample_size) + " λ=" + str(lamda)
    plt.title(plot_title)
    plt.show()


def my_pseudo_inverse(matrix):
    # print("matrix",matrix)
    matrix_transpose = np.transpose(matrix)
    # print("matrix_transpose",matrix_transpose)
    phi_trans_multi_phi = np.dot(matrix_transpose, matrix)
    # print("phi_trans_multi_phi",phi_trans_multi_phi)
    phi_trans_multi_phi_inv = np.linalg.inv(phi_trans_multi_phi)
    # print("phi_trans_multi_phi_inv",phi_trans_multi_phi_inv)
    phi_trans_multi_phi_inv_multi_phi_trans = np.dot(
        phi_trans_multi_phi_inv, matrix_transpose
    )

    return phi_trans_multi_phi_inv_multi_phi_trans


def my_pseudo_inverse_with_regularization(matrix, matrix_dimension, lamda):
    matrix_transpose = np.transpose(matrix)
    phi_trans_multi_phi = np.dot(matrix_transpose, matrix)
    Identity = get_identity_matrix(matrix_dimension)
    regularization_matrix = np.multiply(Identity, lamda)
    phi_trans_multi_phi_add_regularized = np.add(
        phi_trans_multi_phi, regularization_matrix
    )
    phi_trans_multi_phi_inv = np.linalg.inv(phi_trans_multi_phi_add_regularized)
    phi_trans_multi_phi_inv_multi_phi_trans = np.dot(
        phi_trans_multi_phi_inv, matrix_transpose
    )

    return phi_trans_multi_phi_inv_multi_phi_trans


def my_polyfit(big_phi, y, D, lamda=0.0):
    if lamda == 0.0:
        # pseudo_inv_np_big_phi = np.linalg.pinv(np_big_phi)
        # print("pseudo inverse design matrix ", pseudo_inv_np_big_phi)
        my_pseudo_inv_np_big_phi = my_pseudo_inverse(big_phi)
        # print("my pseudo inverse design matrix ", my_pseudo_inv_np_big_phi)
        pseudo_inv_np_big_phi = my_pseudo_inv_np_big_phi
    else:
        # pseudo_inv_np_big_phi = np.linalg.inv(np_big_phi.T @ np_big_phi + lamda * np.eye(np_big_phi.shape[1])) @ np_big_phi.T
        # print("pseudo inverse design matrix ", pseudo_inv_np_big_phi)
        my_regularized_pseudo_inv_np_big_phi = my_pseudo_inverse_with_regularization(
            big_phi, D, lamda
        )
        # print("my regularized pseudo inverse design matrix ", my_regularized_pseudo_inv_np_big_phi)
        pseudo_inv_np_big_phi = my_regularized_pseudo_inv_np_big_phi

    w = np.dot(pseudo_inv_np_big_phi, y)
    # print("w ", w)

    return w


def plot_graphs(train_x, train_y, title, val_x, val_y, test_x, test_y, lamda=0.0):
    # column_names = ['Sample Size', 'Degree', 'λ', 'Train Erms', 'Validate Erms', 'Test Erms']
    # table_df = pd.DataFrame(columns=column_names)

    N = get_N(train_y)
    sample_size = N
    D = N * 0.2
    D = int(D)
    # print("D",D)
    centroids = get_phi_by_gaussian(train_x, D)
    # print("centroids",centroids)

    sigma = estimate_sigma(train_x, centroids)
    # print("sigma=",sigma)

    # print("N = ", N, " d = ", d, " M = ", M, " D = ", D)
    big_phi_train = get_big_phi(train_x, centroids, sigma)
    # print("big_phi_train",big_phi_train)
    w = my_polyfit(big_phi_train, train_y, D, lamda)
    # print("w ", w)

    # print(x1x2)
    # print(new_train_y)
    new_train_y = get_new_y(w, big_phi_train, N, D)
    train_erms = get_erms(new_train_y, train_y)
    # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") train_erms = ",train_erms)

    N = get_N(val_y)
    big_phi_val = get_big_phi(val_x, centroids, sigma)
    new_val_y = get_new_y(w, big_phi_val, N, D)
    validate_erms = get_erms(new_val_y, val_y)
    # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") validate_erms = ",validate_erms)

    N = get_N(test_y)
    big_phi_test = get_big_phi(test_x, centroids, sigma)
    new_test_y = get_new_y(w, big_phi_test, N, D)
    test_erms = get_erms(new_test_y, test_y)
    # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") test_erms = ",test_erms)

    add_data_to_table(sample_size, lamda, train_erms, validate_erms, test_erms)

    plot_scatter(
        train_y, new_train_y, val_y, new_val_y, test_y, new_test_y, title, sample_size, lamda
    )


def main():
    folder_number = "9"
    current_directory = os.getcwd()
    # regression_dataset_2_path=current_directory+ "/Datasets_for_A1/Regression/Dataset 2/" + folder_number + "/"
    regression_dataset_2_path = (
        "/home/dipendu/programs/mtech_2023/ml/ass1/Datasets_for_A1/Regression/Dataset 2/"
        + folder_number
        + "/"
    )

    regression_dataset_2_Train_Sample_1 = (
        regression_dataset_2_path + "train50_" + folder_number + ".csv"
    )
    # df = pd.read_csv(regression_dataset_2_Train_Sample_1)
    # train_x_11 = np.squeeze(np.asanyarray(df[["x1"]]))
    # train_x_12 = np.squeeze(np.asanyarray(df[["x2"]]))
    # train_y_1 = np.squeeze(np.asanyarray(df[["y"]]))
    # data = list(zip(train_x_11, train_x_12, train_y_1))
    # sorted_data = sorted(data, key=lambda x: x[0])
    # train_x_11, train_x_12, train_y_1 = zip(*sorted_data)
    # train_x_1 = []
    # train_x_1.append(train_x_11)
    # train_x_1.append(train_x_12)
    df = pd.read_csv(regression_dataset_2_Train_Sample_1)
    data = df.to_numpy()
    train_x_1 = data[:, :2]
    train_y_1 = data[:, 1]

    regression_dataset_2_Train_Sample_2 = (
        regression_dataset_2_path + "train200_" + folder_number + ".csv"
    )
    # df = pd.read_csv(regression_dataset_2_Train_Sample_2)
    # train_x_21 = np.squeeze(np.asanyarray(df[["x1"]]))
    # train_x_22 = np.squeeze(np.asanyarray(df[["x2"]]))
    # train_y_2 = np.squeeze(np.asanyarray(df[["y"]]))
    # data = list(zip(train_x_21, train_x_22, train_y_2))
    # sorted_data = sorted(data, key=lambda x: x[0])
    # train_x_21, train_x_22, train_y_2 = zip(*sorted_data)
    # train_x_2 = []
    # train_x_2.append(train_x_21)
    # train_x_2.append(train_x_22)
    df = pd.read_csv(regression_dataset_2_Train_Sample_2)
    data = df.to_numpy()
    train_x_2 = data[:, :2]
    train_y_2 = data[:, 1]

    regression_dataset_2_validation = (
        regression_dataset_2_path + "val_" + folder_number + ".csv"
    )
    # df = pd.read_csv(regression_dataset_2_validation)
    # val_x_1 = np.squeeze(np.asanyarray(df[["x1"]]))
    # val_x_2 = np.squeeze(np.asanyarray(df[["x2"]]))
    # val_y = np.squeeze(np.asanyarray(df[["y"]]))
    # data = list(zip(val_x_1, val_x_2, val_y))
    # sorted_data = sorted(data, key=lambda x: x[0])
    # val_x_1, val_x_2, val_y = zip(*sorted_data)
    # val_x = []
    # val_x.append(val_x_1)
    # val_x.append(val_x_2)
    df = pd.read_csv(regression_dataset_2_validation)
    data = df.to_numpy()
    val_x = data[:, :2]
    val_y = data[:, 1]

    regression_dataset_2_test = (
        regression_dataset_2_path + "test_" + folder_number + ".csv"
    )
    # df = pd.read_csv(regression_dataset_2_test)
    # test_x_1 = np.squeeze(np.asanyarray(df[["x1"]]))
    # test_x_2 = np.squeeze(np.asanyarray(df[["x2"]]))
    # test_y = np.squeeze(np.asanyarray(df[["y"]]))
    # data = list(zip(test_x_1, test_x_2, test_y))
    # sorted_data = sorted(data, key=lambda x: x[0])
    # test_x_1, test_x_2, test_y = zip(*sorted_data)
    # test_x = []
    # test_x.append(test_x_1)
    # test_x.append(test_x_2)
    df = pd.read_csv(regression_dataset_2_test)
    data = df.to_numpy()
    test_x = data[:, :2]
    test_y = data[:, 1]    


    regularization_coefficients = [0.0, 0.0001, 1e-9, 1e-18]
    #regularization_coefficients = [0.0]

    for lamda in regularization_coefficients:
        plot_graphs(
            train_x_1,
            train_y_1,
            "train50_" + folder_number,
            val_x,
            val_y,
            test_x,
            test_y,
            lamda,
        )

    for lamda in regularization_coefficients:
        plot_graphs(
            train_x_2,
            train_y_2,
            "train200_" + folder_number,
            val_x,
            val_y,
            test_x,
            test_y,
            lamda,
        )

    print_table()

if __name__ == "__main__":
    main()